In [31]:
from datasets import load_dataset
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import sentencepiece as spm
import csv

In [18]:
data = load_dataset("spider")

Found cached dataset spider (C:/Users/18327/.cache/huggingface/datasets/spider/spider/1.0.0/4e5143d825a3895451569c8b9b55432b91a4bc2d04d390376c950837f4680daa)
100%|██████████| 2/2 [00:00<00:00, 663.29it/s]


In [24]:
query = pd.DataFrame(data["train"]["query"])
question = pd.DataFrame(data["train"]["question"])

In [40]:
# pd.concat([question,query]).to_csv(r'sptrain.txt', header=None, index=None, sep=' ', mode='w')

In [41]:
spm.SentencePieceTrainer.train('--input=sptrain.txt --model_prefix=m --vocab_size=2000')

# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [75]:
t_encode = print(sp.encode("Test Sentence, please encode this and show me your power", out_type=int, enable_sampling=True, alpha=0.1, nbest_size=-1))
print(t_encode)

[3, 45, 129, 17, 3, 9, 130, 17, 130, 137, 75, 24, 3, 114, 705, 3, 130, 960, 46, 75, 3, 177, 87, 26, 3, 208, 46, 3, 26, 1114, 419, 3, 127, 75, 3, 1815, 74, 339, 832, 197]
None


In [80]:
print(sp.decode([3, 45, 129, 17, 3, 9, 130, 17, 130, 137, 75, 24, 3, 114, 705, 3, 130, 960, 46, 75, 3, 177, 87, 26, 3, 208, 46, 3, 26, 1114, 419, 3, 127, 75, 3, 1815, 74, 339, 832, 197]))

Test Sentence, please encode this and show me your power
